### Acceder a Azure Data Lake Storage mediante Token SAS

[Docs](https://learn.microsoft.com/en-us/azure/databricks/connect/storage/azure-storage#access-azure-data-lake-storage-gen2-or-blob-storage-using-a-sas-token)

1. Establecer la configuración de spark "Token SAS"
2. Listar archivos del contenedor "demo".
3. Leer datos del archivo "movie.csv".

Para mount

1. Utiizar el método "mount" de "utility" para montar el almacenamiento
2. Explorar otras utilidades del sistema de archivos relacionados con el montaje (list all mounts, unmounts)

Token sas se genera a nivel de contenedor

El montaje debería hacerse con Entra ID, debido a mis condiciones de espacio de trabajo y no tener acceso a ello, encontré la forma de hacer un montaje con Token SAS, con el código debajo, la diferencia más notable es que se usa el protocolo WASBS en vez de ABFSS

In [0]:
# Definir los parámetros
storage_account = "moviehistorycoursedbc"
container = "demo"
mount_point = f"/mnt/moviehistory/demo"

# Obtener el token SAS desde Azure Key Vault utilizando un Secret Scope
sas_key = dbutils.secrets.get(scope="movie-history-secret-scope", key="movie-access-token-sas-1")

# Intentar desmontar el punto de montaje si ya existe
try:
    dbutils.fs.unmount(mount_point)
    print(f"{mount_point} desmontado exitosamente.")
except Exception as e:
    print(f"No se pudo desmontar {mount_point}. Es posible que no estuviera montado. Detalles: {str(e)}")

# Definir la cadena de origen y la clave de configuración
source_string = f"wasbs://{container}@{storage_account}.blob.core.windows.net/"
conf_key = f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net"

# Intentar montar el contenedor
try:
    dbutils.fs.mount(
        source=source_string,
        mount_point=mount_point,
        extra_configs={conf_key: sas_key}
    )
    print(f"Montaje exitoso en {mount_point}.")
except Exception as e:
    print(f"ERROR: No se pudo montar {mount_point}. Detalles: {str(e)}")


In [0]:
%scala
val storageAccount = "moviehistorycoursedbc"
val container = "demo"
val sasKey = dbutils.secrets.get(scope="movie-history-secret-scope", key="movie-access-token-sas-1")
 
val mountPoint = s"/mnt/moviehistory/demo"
 
 
try {
  dbutils.fs.unmount(s"$mountPoint") // Use this to unmount as needed
} catch {
  case ioe: java.rmi.RemoteException => println(s"$mountPoint already unmounted")
}
 
 
val sourceString = s"wasbs://$container@$storageAccount.blob.core.windows.net/"
val confKey = s"fs.azure.sas.$container.$storageAccount.blob.core.windows.net"
 
 
 
try {
  dbutils.fs.mount(
    source = sourceString,
    mountPoint = mountPoint,
    extraConfigs = Map(confKey -> sasKey)
  )
}
catch {
  case e: Exception =>
    println(s"*** ERROR: Unable to mount $mountPoint. Run previous cells to unmount first")
}

In [0]:
display(dbutils.fs.ls("/mnt/moviehistory/demo"))

In [0]:
display(spark.read.csv("/mnt/moviehistory/demo/movie.csv", header=True))

In [0]:
display(dbutils.fs.mounts())

In [0]:
dbutils.fs.unmount("/mnt/moviehistory/demo")